In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import torchvision.utils as utils
import os

In [2]:
#dataset path
data_path_train = "./data/training"
data_path_test = "./data/testing"

In [3]:
# data transform, you can add different transform methods and resize image to any size
img_size = 224
transform = transforms.Compose([
                       transforms.Resize((img_size,img_size)),
                       transforms.ToTensor()
                       ])

#build dataset
dataset = datasets.ImageFolder(root=data_path_train,transform=transform)

# spilt your data into train and val
TOTAL_SIZE = len(dataset)
ratio = 0.9
train_len = round(TOTAL_SIZE * ratio)
valid_len = round(TOTAL_SIZE * (1-ratio))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, valid_len])

#build dataloader
train_data_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=True,  num_workers=4)

#check dataset
print(dataset)
print(dataset.class_to_idx)

Dataset ImageFolder
    Number of datapoints: 1646
    Root location: ./data/training
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )
{'Baked Potato': 0, 'Crispy Chicken': 1, 'Donut': 2, 'Fries': 3}


/home/NFS/opt/anaconda3/envs/ml2023f/lib/python3.8/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
#train function
def train(model, criterion, optimizer):
    model.train()
    total_loss = 0.0
    total_correct = 0
    
    # Iterate over data
    for inputs, labels in train_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad() # Be sure to add and thoroughly clean the previous gradient.

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backward + optimize
        loss.backward()  # Get gradient of the whole model by loss function
        optimizer.step() # Update weight

        # statistics
        total_loss += loss.item()
        total_correct += torch.sum(preds == labels.data)
        
    avg_loss = total_loss / len(train_data_loader)
    accuracy = total_correct.double() / len(train_dataset) * 100

    print('Training Accuracy: {:.4f}% Training Loss: {:.4f}'.format(accuracy, avg_loss))
    return 

#validation function
def valid(model, criterion):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    
    # Iterate over data
    for inputs, labels in val_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        total_loss += loss.item()
        total_correct += torch.sum(preds == labels.data)
        
    avg_loss = total_loss / len(val_data_loader)
    accuracy = total_correct.double() / len(val_dataset) * 100

    print('Validation Accuracy: {:.4f}% Validation Loss: {:.4f}'.format(accuracy, avg_loss))
    return accuracy

In [5]:
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#build your model here
class RN18BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(RN18BasicBlock, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels , out_channels, kernel_size = 3, stride = stride, padding = 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(out_channels)
        )

        self.extra = nn.Sequential()
        if in_channels != out_channels:
            self.extra = nn.Sequential(
                nn.Conv2d(in_channels , out_channels, kernel_size = 1, stride = stride),
                nn.BatchNorm2d(out_channels),
            )
        
    def forward(self, x):
        out  = self.layer(x)
        out2 = self.extra(x)
        return F.relu(out + out2)

class RN18(nn.Module):
    def __init__(self):
        super(RN18, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.conv2 = nn.Sequential(
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
            RN18BasicBlock(64,64,1),
            RN18BasicBlock(64,64,1)
        )

        self.conv3 = nn.Sequential(
            RN18BasicBlock(64 ,128,2),
            RN18BasicBlock(128,128,1)
        )

        self.conv4 = nn.Sequential(
            RN18BasicBlock(128,256,2),
            RN18BasicBlock(256,256,1)
        )

        self.conv5 = nn.Sequential(
            RN18BasicBlock(256,512,2),
            RN18BasicBlock(512,512,1)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512,4)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = self.avgpool(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

#call model
model = RN18()


In [6]:
####################  implement your optimizer ###################################
## you can use any training methods if you want (ex:lr decay, weight decay.....)
learning_rate = 0.001
epochs = 50
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas = (0.99,0.999), weight_decay = 0.00001)
criterion = nn.CrossEntropyLoss()


# start training
model.to(device=device)
acc_best = 0.0

print('--------------start training--------------')
for epoch in range(1, epochs+1):
    
    print('epoch:', epoch)
    train(model, criterion, optimizer)
    accuracy = valid(model, criterion)
    
    if accuracy >= acc_best:
        acc_best = accuracy
        print("model saved")
        # save the model
        torch.save(model, "model.pth")
    learning_rate *= 0.9

--------------start training--------------
epoch: 1
Training Accuracy: 48.0081% Training Loss: 1.3345
Validation Accuracy: 49.6970% Validation Loss: 1.4137
model saved
epoch: 2
Training Accuracy: 60.4997% Training Loss: 0.9528
Validation Accuracy: 44.8485% Validation Loss: 1.4230
epoch: 3
Training Accuracy: 63.3356% Training Loss: 0.9478
Validation Accuracy: 54.5455% Validation Loss: 1.1062
model saved
epoch: 4
Training Accuracy: 65.4963% Training Loss: 0.8397
Validation Accuracy: 63.0303% Validation Loss: 0.9241
model saved
epoch: 5
Training Accuracy: 70.3579% Training Loss: 0.7815
Validation Accuracy: 61.8182% Validation Loss: 1.0401
epoch: 6
Training Accuracy: 70.1553% Training Loss: 0.7767
Validation Accuracy: 60.6061% Validation Loss: 1.1955
epoch: 7
Training Accuracy: 72.0459% Training Loss: 0.7132
Validation Accuracy: 67.8788% Validation Loss: 0.7812
model saved
epoch: 8
Training Accuracy: 74.6793% Training Loss: 0.6662
Validation Accuracy: 68.4848% Validation Loss: 0.8321
model